Import the libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
import re
import os

# Warnings filtering.
import warnings
warnings.filterwarnings('ignore')

Read the datasets

In [2]:
data_path1 = (r"C:\Users\black\vanilla-steel-assessment\data\supplier_data1.xlsx")
data_path2 = (r"C:\Users\black\vanilla-steel-assessment\data\supplier_data2.xlsx")

In [3]:
raw_supplier1 = pd.read_excel(data_path1)
raw_supplier2 = pd.read_excel(data_path2)

In [4]:
raw_supplier1.head()

,Quality/Choice,Grade,Finish,Thickness (mm),Width (mm),Description,Gross weight (kg),RP02,RM,Quantity,AG,AI
0,3rd,C200S,gebeizt und geglüht,2.77,1100,Längs- oder Querisse,13983,333.6,606.2,0.00,16.11,0.0054
1,3rd,C300S,ungebeizt,2.65,1075,Längs- oder Querisse,13047,717.7,0.0,0.00,16.11,0.0046
2,3rd,C100S,gebeizt und geglüht,2.20,1100,Kantenfehler - FS-Kantenrisse,14155,368.9,0.0,10.84,0.00,0.0061
3,2nd,C100S,gebeizt,2.86,1100,Längs- oder Querisse,11381,368.9,601.7,22.87,0.00,0.0062
4,1st,C300S,ungebeizt,2.88,1050,Sollmasse (Gewicht) unterschritten,10072,0.0,1213.0,22.87,0.00,0.0041


supplier1

In [ ]:
# Rename columns to clean english names
rename_map = {
    'Quality/Choice': 'quality',
    'Grade': 'grade',
    'Finish': 'finish',
    'Thickness (mm)': 'thickness_mm',
    'Width (mm)': 'width_mm',
    'Description': 'description',
    'Gross weight (kg)': 'weight_kg',
    'Quantity': 'quantity',
    'RP02': 'rp02',
    'RM': 'rm',
    'AG': 'ag',
    'AI': 'ai'
}

raw_supplier1.rename(columns=rename_map, inplace=True)

In [ ]:
# Convert numeric fields
numeric_cols = ['thickness_mm', 'width_mm', 'gross_weight_kg', 'quantity', 'rp02', 'rm', 'ag', 'ai']

for col in numeric_cols:
    if col in raw_supplier1.columns:
        raw_supplier1[col] = (
            raw_supplier1[col]
            .astype(str)
            .str.replace(",", ".", regex=False)   # Fix comma decimals 
            .str.extract(r"([\d.]+)")             # Extract numeric part 
            .astype(float)
        )

In [ ]:
raw_supplier1["data_source"] = "supplier1"

In [8]:
# Preview
raw_supplier1.head()

,quality,grade,finish,thickness_mm,width_mm,description,weight_kg,rp02,rm,quantity,ag,ai,data_source
0,3rd,C200S,gebeizt und geglüht,2.77,1100.0,Längs- oder Querisse,13983,333.6,606.2,0.00,16.11,0.0054,supplier1
1,3rd,C300S,ungebeizt,2.65,1075.0,Längs- oder Querisse,13047,717.7,0.0,0.00,16.11,0.0046,supplier1
2,3rd,C100S,gebeizt und geglüht,2.20,1100.0,Kantenfehler - FS-Kantenrisse,14155,368.9,0.0,10.84,0.00,0.0061,supplier1
3,2nd,C100S,gebeizt,2.86,1100.0,Längs- oder Querisse,11381,368.9,601.7,22.87,0.00,0.0062,supplier1
4,1st,C300S,ungebeizt,2.88,1050.0,Sollmasse (Gewicht) unterschritten,10072,0.0,1213.0,22.87,0.00,0.0041,supplier1


supplier2

In [9]:
# Rename columns to match our unified format
rename_map = {
    'Material': 'material',
    'Description': 'description',
    'Article ID': 'article_id',
    'Weight (kg)': 'weight_kg',
    'Quantity': 'quantity',
    'Reserved': 'reserved'
}

raw_supplier2.rename(columns=rename_map, inplace=True)

In [ ]:
# Clean numeric columns
numeric_cols = ['gross_weight_kg', 'quantity']

for col in numeric_cols:
    if col in raw_supplier2.columns:
        raw_supplier2[col] = (
            raw_supplier2[col]
            .astype(str)
            .str.replace(",", ".", regex=False)
            .str.extract(r"([\d.]+)")
            .astype(float)
        )

In [11]:
# Normalize reserved column
raw_supplier2["reserved"] = raw_supplier2["reserved"].astype(str).str.lower().map({
    "true": True,
    "yes": True,
    "false": False,
    "no": False
}).fillna(False)

In [ ]:
raw_supplier2["data_source"] = "supplier2"

In [13]:
raw_supplier2.head()

,material,description,article_id,weight_kg,quantity,reserved,data_source
0,HDC,Material is Oiled,23048203,24469,52.0,False,supplier2
1,S235JR,Material is Oiled,23040547,16984,41.0,False,supplier2
2,S235JR,Material is Painted,23046057,9162,28.0,False,supplier2
3,DX51D +AZ150,Material is Oiled,23041966,12119,66.0,False,supplier2
4,HDC,Material is Painted,23043884,17260,26.0,False,supplier2


In [14]:
raw_supplier1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   quality       50 non-null     object 
 1   grade         50 non-null     object 
 2   finish        50 non-null     object 
 3   thickness_mm  50 non-null     float64
 4   width_mm      50 non-null     float64
 5   description   50 non-null     object 
 6   weight_kg     50 non-null     int64  
 7   rp02          50 non-null     float64
 8   rm            50 non-null     float64
 9   quantity      50 non-null     float64
 10  ag            50 non-null     float64
 11  ai            50 non-null     float64
 12  data_source   50 non-null     object 
dtypes: float64(7), int64(1), object(5)
memory usage: 5.2+ KB


In [15]:
raw_supplier1.duplicated().sum()

np.int64(0)

In [16]:
raw_supplier2.duplicated().sum()

np.int64(0)

In [17]:
raw_supplier1.isnull().sum()

quality         0
grade           0
finish          0
thickness_mm    0
width_mm        0
description     0
weight_kg       0
rp02            0
rm              0
quantity        0
ag              0
ai              0
data_source     0
dtype: int64

In [18]:
#unique values per column in supplier1
for col in raw_supplier1.columns:
    print(f"\n{col} ({raw_supplier1[col].nunique(dropna=False)} unique):")
    print(raw_supplier1[col].unique())


quality (3 unique):
['3rd' '2nd' '1st']

grade (3 unique):
['C200S' 'C300S' 'C100S']

finish (3 unique):
['gebeizt und geglüht' 'ungebeizt' 'gebeizt']

thickness_mm (39 unique):
[2.77 2.65 2.2  2.86 2.88 2.51 2.69 2.98 2.03 2.54 2.31 2.55 2.52 2.46
 2.74 2.27 2.08 2.29 2.19 2.14 2.84 2.22 2.83 2.32 2.48 2.21 2.75 2.89
 2.53 2.18 2.43 2.82 2.8  2.01 2.3  2.12 2.37 2.72 2.79]

width_mm (5 unique):
[1100. 1075. 1050. 1000. 1150.]

description (3 unique):
['Längs- oder Querisse' 'Kantenfehler - FS-Kantenrisse'
 'Sollmasse (Gewicht) unterschritten']

weight_kg (50 unique):
[13983 13047 14155 11381 10072 12863  8936  9810  5491 11415 14758 13400
 12787  8391 10409  5951  8240 14139  7304 13518 12010  6665 14839 10959
 14688  5979 10499 10723 10262  8118  5011  8410  5124 12071 13321  7968
 12702  5915  8047  7772  9995  9774 11360  8261  9110 13667  6947  8786
 14311  8048]

rp02 (4 unique):
[333.6 717.7 368.9   0. ]

rm (4 unique):
[ 606.2    0.   601.7 1213. ]

quantity (4 unique):
[ 0.  

In [19]:
raw_supplier2.isnull().sum()

material       0
description    0
article_id     0
weight_kg      0
quantity       0
reserved       0
data_source    0
dtype: int64

In [20]:
raw_supplier2.head()

,material,description,article_id,weight_kg,quantity,reserved,data_source
0,HDC,Material is Oiled,23048203,24469,52.0,False,supplier2
1,S235JR,Material is Oiled,23040547,16984,41.0,False,supplier2
2,S235JR,Material is Painted,23046057,9162,28.0,False,supplier2
3,DX51D +AZ150,Material is Oiled,23041966,12119,66.0,False,supplier2
4,HDC,Material is Painted,23043884,17260,26.0,False,supplier2


In [21]:
raw_supplier2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   material     50 non-null     object 
 1   description  50 non-null     object 
 2   article_id   50 non-null     int64  
 3   weight_kg    50 non-null     int64  
 4   quantity     50 non-null     float64
 5   reserved     50 non-null     bool   
 6   data_source  50 non-null     object 
dtypes: bool(1), float64(1), int64(2), object(3)
memory usage: 2.5+ KB


In [22]:
#unique values per column in supplier1
for col in raw_supplier2.columns:
    print(f"\n{col} ({raw_supplier2[col].nunique(dropna=False)} unique):")
    print(raw_supplier2[col].unique())


material (4 unique):
['HDC' 'S235JR' 'DX51D +AZ150' 'DX51D +Z140']

description (3 unique):
['Material is Oiled' 'Material is Painted' 'Material is not Oiled']

article_id (50 unique):
[23048203 23040547 23046057 23041966 23043884 23048868 23046459 23043788
 23044935 23049059 23047860 23047939 23047348 23045961 23042649 23048176
 23043667 23046190 23044291 23046841 23041008 23041560 23043638 23041902
 23049627 23044610 23049106 23048028 23045099 23046741 23047239 23049907
 23044447 23047489 23040273 23040905 23043967 23045724 23049672 23049617
 23043836 23049333 23041897 23040756 23041086 23047543 23045292 23047408
 23043348 23043386]

weight_kg (50 unique):
[24469 16984  9162 12119 17260 23258 20994 22325 10571 12999 15862  9217
 15881 23915 15741 15717 23967 21346 10389 18501 14277 24951 15121 17933
 20304 20027 23131  6113  9037 24331 11898 21609 19165 18716  7025 14796
 21149 16393 21321 22395  9355 10143 17001  6034 15230  9940 23884  5302
 16927 11191]

quantity (38 unique):
[52

In [23]:
print("Supplier1 columns:", list(raw_supplier1.columns))
print("Supplier2 columns:", list(raw_supplier2.columns))

Supplier1 columns: ['quality', 'grade', 'finish', 'thickness_mm', 'width_mm', 'description', 'weight_kg', 'rp02', 'rm', 'quantity', 'ag', 'ai', 'data_source']
Supplier2 columns: ['material', 'description', 'article_id', 'weight_kg', 'quantity', 'reserved', 'data_source']


In [24]:
print("shape of supplier1:", raw_supplier1.shape)
print("shape of supplier2:", raw_supplier2.shape)

shape of supplier1: (50, 13)
shape of supplier2: (50, 7)


# combining the dataframes 

In [25]:
# Define common columns
common_columns = [
    "material", "grade", "finish",
    "thickness_mm", "width_mm", "description", "weight_kg",
    "quantity", "source"
]

# Add missing columns to supplier1
raw_supplier1["material"] = np.nan
raw_supplier1["reserved"] = np.nan
raw_supplier1["source"] = "supplier1"

# Step 3: Add missing columns to supplier2
raw_supplier2["grade"] = np.nan
raw_supplier2["finish"] = np.nan
raw_supplier2["thickness_mm"] = np.nan
raw_supplier2["width_mm"] = np.nan
raw_supplier2["source"] = "supplier2"

# Reorder both DataFrames
supplier1 = raw_supplier1[common_columns]
supplier2 = raw_supplier2[common_columns]

# Combine them into one dataset
raw_inventory_dataset = pd.concat([supplier1, supplier2], ignore_index=True)

In [26]:
final_columns = [
    "material", "grade", "finish", "thickness_mm", "width_mm",
    "description", "weight_kg", "quantity", "source"
]

inventory_dataset = raw_inventory_dataset[final_columns]

In [27]:
# Percentage of missing values per column
missing_percentage = inventory_dataset.isnull().sum() * 100 / len(inventory_dataset)

# Display
print(missing_percentage.sort_values(ascending=False))


material        50.0
grade           50.0
finish          50.0
thickness_mm    50.0
width_mm        50.0
description      0.0
weight_kg        0.0
quantity         0.0
source           0.0
dtype: float64


In [ ]:
# Fill all missing values explicitly with NaN
inventory_dataset = inventory_dataset.fillna(np.nan)

In [29]:
inventory_dataset

,material,grade,finish,thickness_mm,width_mm,description,weight_kg,quantity,source
0,NaN,C200S,gebeizt und geglüht,2.77,1100.0,Längs- oder Querisse,13983,0.00,supplier1
1,NaN,C300S,ungebeizt,2.65,1075.0,Längs- oder Querisse,13047,0.00,supplier1
2,NaN,C100S,gebeizt und geglüht,2.20,1100.0,Kantenfehler - FS-Kantenrisse,14155,10.84,supplier1
3,NaN,C100S,gebeizt,2.86,1100.0,Längs- oder Querisse,11381,22.87,supplier1
4,NaN,C300S,ungebeizt,2.88,1050.0,Sollmasse (Gewicht) unterschritten,10072,22.87,supplier1
...,...,...,...,...,...,...,...,...,...
95,DX51D +Z140,NaN,NaN,NaN,NaN,Material is not Oiled,9940,96.00,supplier2
96,DX51D +AZ150,NaN,NaN,NaN,NaN,Material is Painted,23884,51.00,supplier2
97,S235JR,NaN,NaN,NaN,NaN,Material is Oiled,5302,12.00,supplier2
98,DX51D +Z140,NaN,NaN,NaN,NaN,Material is Oiled,16927,69.00,supplier2


In [30]:
from pathlib import Path

# Start from current notebook location
notebook_dir = Path().resolve()

# Go up two levels (from scripts/notebooks/ to project root)
project_root = notebook_dir.parents[1]

# Define output folder at top-level
output_dir = project_root / "output"
output_dir.mkdir(parents=True, exist_ok=True)

# Save file
inventory_dataset.to_csv(output_dir / "inventory_dataset.csv", index=False)
print(f"Saved to: {output_dir / 'inventory_dataset.csv'}")

Saved to: C:\Users\black\vanilla-steel-assessment\output\inventory_dataset.csv
